# Divergent data reduction for Amor

In this notebook, we will look at how to use the `essreflectometry` package with Sciline, for reflectometry data collected from the PSI instrument [Amor](https://www.psi.ch/en/sinq/amor) in [divergent beam mode](https://www.psi.ch/en/sinq/amor/selene).

We will begin by importing the modules that are necessary for this notebook.

In [ ]:
import scipp as sc
import sciline
from essreflectometry.amor import providers, default_parameters
from essreflectometry.types import *

In [ ]:
params={
    **default_parameters,
    QBins: sc.geomspace(dim='Q', start=0.008, stop=0.075, num=200, unit='1/angstrom'),
    SampleRotation[Sample]: sc.scalar(0.7989, unit='deg'),
    Filename[Sample]: "sample.nxs",
    SampleRotation[Reference]: sc.scalar(0.8389, unit='deg'),
    Filename[Reference]: "reference.nxs",
}

In [ ]:
pipeline = sciline.Pipeline(
    providers,
    params=params
)

In [ ]:
pipeline.visualize((NormalizedIofQ, QResolution), graph_attr={'rankdir': 'LR'})

In [ ]:
# Compute I over Q and the standard deviation of Q
ioq, qstd = pipeline.compute((NormalizedIofQ, QResolution)).values()

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5, 7))
ax1 = fig.add_axes([0, 0.55, 1.0, 0.45])
ax2 = fig.add_axes([0, 0.0, 1.0, 0.45])
cax = fig.add_axes([1.05, 0.55, 0.03, 0.45])
fig1 = ioq.plot(norm='log', ax=ax1, cax=cax, grid=True)
fig2 = ioq.mean('detector_number').plot(norm='log', ax=ax2, grid=True)
fig1.canvas.xrange = fig2.canvas.xrange

## Make a $(\lambda, \theta)$ map
A good sanity check is to create a two-dimensional map of the counts in $\lambda$ and $\theta$ bins. To achieve this, we request the `ThetaData` from the pipeline. In the graph above we can see that `WavelengthData` is required to compute `ThetaData`, therefore it is also present in `ThetaData` so we don't need to require it separately.

In [ ]:
from essreflectometry.types import ThetaData
pipeline.compute(ThetaData[Sample])\
    .bins.concat('detector_number')\
    .hist(
        theta=sc.linspace(dim='theta', start=0.0, stop=1.2, num=165, unit='deg').to(unit='rad'),
        wavelength=sc.linspace(dim='wavelength', start=0, stop=15.0, num=165, unit='angstrom'),
    )\
    .plot()


This plot can be used to check if the value of the sample rotation angle $\omega$ is correct. The bright triangles should be pointing back to the origin $\lambda = \theta = 0$.